In [3]:
import pandas as pd
import numpy as np
from data_preprocessing import *

set_names = ['INDU-HAND-RAND',
             'SAT_Competition_RACE_HAND',
             'SAT_Competition_RACE_RAND',
             'SAT_Competition_RACE_INDU',
             'IBM-SWV',
             'IBM-ALL',
             'SWV',
             'SAT_Competition_RACE_RAND_SAT']

# ovo je skup za koji smo diskutovali visoku vrednost varijanse objasnjene jednom PCA komponentom
set_name = set_names[0] 

X = pd.read_csv('data/' + 'SAT_Competition_RACE_RAND_SAT' + '-feat.csv')
Y = pd.read_csv('data/' + 'SAT_Competition_RACE_RAND_SAT' + '-' + 'tnm' + '.csv',
                  names=['INSTANCE_ID', 'SOLVER_TIME'])
print(X['INSTANCE_ID'].equals(Y['INSTANCE_ID']))

X = pd.read_csv('data/gcrf/' + 'INDU-IBM-SWV' + '-feat.csv')
Y = pd.read_csv('data/gcrf/' + 'INDU-IBM-SWV' + '-' + 'spear' + '.csv',
                  names=['INSTANCE_ID', 'SOLVER_TIME'])
print(X['INSTANCE_ID'].equals(Y['INSTANCE_ID']))
X = X.drop('INSTANCE_ID', axis=1).get_values()

# print(X.shape)
# X[X==512]

True
True


In [4]:
np.linalg.cond(X.T.dot(X))

1.883070706753194e+32

In [5]:
# uslovljenosti svih matrica
for set_name in set_names:
    X = pd.read_csv('data/' + set_name + '-feat.csv')
    X = X.drop('INSTANCE_ID', axis=1).get_values()
    print(set_name, ':', np.linalg.cond(X.T.dot(X)))

INDU-HAND-RAND : 1.1022481357927698e+21
SAT_Competition_RACE_HAND : 1.4277627953673724e+21
SAT_Competition_RACE_RAND : 2.2969465109837296e+29
SAT_Competition_RACE_INDU : 6.426270817861262e+31
IBM-SWV : 1.154136200553743e+32
IBM-ALL : 6.162783605409098e+33
SWV : 1.0733666398452457e+35
SAT_Competition_RACE_RAND_SAT : 1.8748828491341977e+29


In [6]:
# broj instanci pre i nakon izbacivanja onih sa ostecenim vrednostima atributa
for set_name in set_names:
    X = pd.read_csv('data/' + set_name + '-feat.csv')
    X = X.drop('INSTANCE_ID', axis=1).get_values()
    print(set_name, ':', X.shape[0], X[~(X == -512).any(axis=1)].shape[0])

INDU-HAND-RAND : 7012 2802
SAT_Competition_RACE_HAND : 1955 765
SAT_Competition_RACE_RAND : 3381 1867
SAT_Competition_RACE_INDU : 1676 170
IBM-SWV : 1369 22
IBM-ALL : 765 8
SWV : 604 14
SAT_Competition_RACE_RAND_SAT : 2076 1140


In [34]:
def S(Y, delta=1, svd=False, scale=False, drop=3):
  num_solvers = Y.shape[1]
  S = np.zeros((num_solvers, num_solvers))
  if svd == True:
    U, _sigma, _V = np.linalg.svd(Y.T)
    U = U[:, :-drop]
  min = max = 0
  for k in range(0, num_solvers):
    for l in range(0, num_solvers):
      if k == l: continue
      if svd == True:
#         print(U[k, :])
#         print(U[l, :])
#         print(np.linalg.norm(U[k, :] - U[l, :]) ** 2)
#         print("\n")
        S[k, l] = np.exp(- delta * np.linalg.norm(U[k, :] - U[l, :]) ** 2)
      else:
#         print(np.linalg.norm(Y[:, k] - Y[:, l]) ** 2)
#         print("\n")
        S[k, l] = np.exp(- delta * np.linalg.norm(Y[:, k] - Y[:, l]) ** 2)
      S[l, k] = S[k, l]
      if S[k, l] > max: max = S[k, l]
  if scale:
    print(min)
    print(max)
    # scaling to [0, 1]
    if max > 0: S = (S - min) / (max - min)
  return S

In [35]:
# dataset = 'RAND_SAT'
# dataset = 'INDU-IBM-SWV'
# Y = pd.read_csv('data/gcrf/' + dataset + '-results.csv').drop('INSTANCE_ID', axis=1).get_values()
data = pd.read_csv('SATzilla2012_data/SATHAND12S.csv')
solver_times = data.filter(regex='_Time$', axis=1).get_values()
features = data.drop(data.iloc[:, 0:156], axis=1)
# features


In [62]:
S(solver_times, delta=1.9, svd=True, scale=True, drop=30)

0
0.9999999942666387


array([[0.        , 0.99997419, 0.99992134, 0.99998704, 0.99997342,
        0.9999888 , 0.99999984, 0.99903947, 0.99987298, 0.99993208,
        0.99994526, 0.99997533, 0.99991566, 0.99956376, 0.99988712,
        0.998825  , 0.99999601, 0.99994649, 0.99892609, 0.99999691,
        0.99997154, 0.99990511, 0.99969643, 0.99999742, 0.9999979 ,
        0.99950613, 0.99976926, 0.9999906 , 0.99881932, 1.        ,
        0.9997833 ],
       [0.99997419, 0.        , 0.99998565, 0.99999781, 1.        ,
        0.99992898, 0.99997817, 0.99932842, 0.99996169, 0.99999001,
        0.99984427, 0.99999999, 0.99998317, 0.99932582, 0.99975336,
        0.99845128, 0.99994988, 0.99984634, 0.99923308, 0.99995323,
        0.99989151, 0.99997828, 0.99949363, 0.99995529, 0.99995734,
        0.99970609, 0.99989779, 0.99999595, 0.99914244, 0.99997331,
        0.99990706],
       [0.99992134, 0.99998565, 0.        , 0.99997224, 0.99998622,
        0.99985077, 0.9999284 , 0.99951036, 0.99999424, 0.99999961,
      

## 